In [1]:
!pip3 install nltk

In [2]:
!curl https://raw.githubusercontent.com/codelucas/newspaper/master/download_corpora.py | python3

Finished.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:07 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0
  0     0    0     0    0     0      0      0 --:--

In [3]:
import nltk
nltk.download('punkt')  # Downloads the 'punkt' tokenizer models


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
!pip3 show newspaper3k

Name: newspaper3k
Version: 0.2.8
Summary: Simplified python article discovery & extraction.
Home-page: https://github.com/codelucas/newspaper/
Author: Lucas Ou-Yang
Author-email: lucasyangpersonal@gmail.com
License: MIT
Location: C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: beautifulsoup4, cssselect, feedfinder2, feedparser, jieba3k, lxml, nltk, Pillow, python-dateutil, PyYAML, requests, tinysegmenter, tldextract
Required-by: 


In [5]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
# import newspaper
from newspaper import Article

from bs4 import BeautifulSoup
import urllib.request
import nltk
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

In [20]:
# VNEXPRESS
def get_page_urls_vnexpress(base_url, quantity):
    news_per_page = 25
    if quantity % news_per_page == 0:
        no_of_pages = quantity // news_per_page
    else:
        no_of_pages = quantity // news_per_page + 1

    extend_url = ['-p{}'.format(page) for page in range(2, no_of_pages + 1)]
    url_list = [base_url + extend_part for extend_part in extend_url]
    url_list.insert(0, base_url)
    return url_list

# VNEXPRESS 
def get_links_in_page_vnexpress(url):
    try:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page, 'html.parser')
        # <section>
        sections = soup.find_all('section', attrs={'class':'section section_container mt15'})
        # <h3>
        h3_all = []
        for section in sections:
            h3_all.extend(section.find_all('h3', attrs={'class':'title-news'}))
        # <a>
        a_all = []
        results = [[]]
        for h3 in h3_all:
            a_all.extend(h3.find_all('a', attrs={'class':''}))
        for a in a_all:
            title = a.get('title')
            link = a.get('href')
            print('Title: {} - Link: {}'.format(title, link))
            results.append([title,link])
        return results
    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e.code} for URL: {url}")
        return []
    


def crawl_by_url(url, title):
    try:
        article = Article(url, language='vi')
        article.download()
        article.parse()
        contents = str(article.text).replace("\n","")
        article.nlp()
        
        # replace
        char_to_replace = {':': '',
                          '/': '',
                          '\\': '',
                          '*': '',
                          '?': '',
                          '"': '',
                          '<': '',
                          '>': '',
                          '|': ''}
        
        for key, value in char_to_replace.items():
            # Replace key character with value character in string
            title = title.replace(key, value)
        
        # write txt
        # https://vnexpress.net
        if(url.find('https://vnexpress.net/') != -1):
            output = "data/text/vnexpress/" + title  + ".txt"
            if not os.path.exists(output):  # Kiểm tra xem file đã tồn tại chưa
                with open(output, "w+", encoding='utf-8') as f:
                    f.write(title+"\n")
                    f.write(contents)
        elif(url.find('https://thanhnien.vn/') != -1):
            output = "data/text/thanhnien/" + title  + ".txt"
            if not os.path.exists(output):  # Kiểm tra xem file đã tồn tại chưa
                with open(output, "w+", encoding='utf-8') as f:
                    f.write(str(url)+"\n")
                    f.write(title+"\n")
                    f.write(contents)
        elif(url.find('https://vietnamnet.vn/') != -1):
            output = "data/text/vietnamnet/" + title  + ".txt"
            if not os.path.exists(output):  # Kiểm tra xem file đã tồn tại chưa
                with open(output, "w+", encoding='utf-8') as f:
                    # f.write(str(url)+"\n")
                    f.write(title+"\n")
                    f.write(contents)
    except Exception as e:
        print(f"Error processing URL: {url} - {e}")

def vnexpress(base_url):
    page_urls = get_page_urls_vnexpress(base_url, quantity=500)
#     print(page_urls)
    total_articles = 0
    for page in page_urls:
        arr = get_links_in_page_vnexpress(page)
        for i in range(1,len(arr)-1):
            title = arr[i][0]
            url = arr[i][1]
            crawl_by_url(url,title)

# base_url = 'http://vnexpress.net/thoi-su' ## category = thoi-su
# Chạy thêm theo danh sách các thể loại
categories = ['giai-tri', 'thoi-su']
# categories = ['thoi-su']
# categories = ['giai-tri']

for category in reversed(categories):
    vnexpress(f'http://vnexpress.net/{category}')

Title: Ông Đồng Văn Thanh được bầu Bí thư Tỉnh ủy Hậu Giang - Link: https://vnexpress.net/ong-dong-van-thanh-duoc-bau-bi-thu-tinh-uy-hau-giang-4822528.html
Title: Chủ tịch Quốc hội: Đổi mới lập pháp, giao quyền chủ động cho Chính phủ - Link: https://vnexpress.net/chu-tich-quoc-hoi-doi-moi-lap-phap-giao-quyen-chu-dong-cho-chinh-phu-4822469.html
Title: Miền Bắc sắp đón không khí lạnh nhất từ đầu mùa - Link: https://vnexpress.net/mien-bac-sap-don-khong-khi-lanh-nhat-tu-dau-mua-4822484.html
Title: Động đất làm đá lăn xuống gần khu dân cư - Link: https://vnexpress.net/dong-dat-lam-da-lan-xuong-gan-khu-dan-cu-4822462.html
Title: Cháy khách sạn 7 tầng ở Hà Nội, 20 người thoát nạn - Link: https://vnexpress.net/chay-khach-san-7-tang-o-ha-noi-20-nguoi-thoat-nan-4822459.html
Title: Thủ tướng: Tăng tốc phát triển kinh tế xã hội và sắp xếp bộ máy - Link: https://vnexpress.net/thu-tuong-tang-toc-phat-trien-kinh-te-xa-hoi-va-sap-xep-bo-may-4822432.html
Title: Tổng Bí thư: Tinh gọn bộ máy không thể ch

In [ ]:
# VIETNAMNET
def get_page_urls_vietnamnet(base_url, quantity):
    news_per_page = 25
    if quantity % news_per_page == 0:
        no_of_pages = quantity // news_per_page
    else:
        no_of_pages = quantity // news_per_page + 1

    extend_url = ['-p{}'.format(page) for page in range(2, no_of_pages + 1)]
    url_list = [base_url + extend_part for extend_part in extend_url]
    url_list.insert(0, base_url)
    return url_list

# VIETNAMNET
def get_links_in_page_vietnamnet(url):
    try:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page, 'html.parser')
        # <div>
        divs = soup.find_all('div', attrs={'class':'sectionTopstory sectionTopstory__left'})
        # <h3>
        h3_all = []
        for section in divs:
            h3_all.extend(section.find_all('h3', attrs={'class':'horizontalPost__main-title vnn-title'}))
        # <a>
        a_all = []
        results = [[]]
        for h3 in h3_all:
            a_all.extend(h3.find_all('a'))
        for a in a_all:
            title = a.get('title')
            link = a.get('href')
            print('Title: {} - Link: {}'.format(title, link))
            results.append([title,link])
            print (results)
        return results
        
    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e.code} for URL: {url}")
        return []
    


def crawl_by_url(url, title):
    try:
        article = Article(url, language='vi')
        article.download()
        article.parse()
        contents = str(article.text).replace("\n","")
        article.nlp()
        
        # replace
        char_to_replace = {':': '',
                          '/': '',
                          '\\': '',
                          '*': '',
                          '?': '',
                          '"': '',
                          '<': '',
                          '>': '',
                          '|': ''}
        
        for key, value in char_to_replace.items():
            # Replace key character with value character in string
            title = title.replace(key, value)
        
        # write txt
        # https://vnexpress.net
        if(url.find('https://vnexpress.net/') != -1):
            output = "data/text/vnexpress/" + title  + ".txt"
            if not os.path.exists(output):  # Kiểm tra xem file đã tồn tại chưa
                with open(output, "w+", encoding='utf-8') as f:
                    f.write(title+"\n")
                    f.write(contents)
        elif(url.find('https://thanhnien.vn/') != -1):
            output = "data/text/thanhnien/" + title  + ".txt"
            if not os.path.exists(output):  # Kiểm tra xem file đã tồn tại chưa
                with open(output, "w+", encoding='utf-8') as f:
                    f.write(str(url)+"\n")
                    f.write(title+"\n")
                    f.write(contents)
        elif(url.find('https://vietnamnet.vn/') != -1):
            output = "data/text/vietnamnet/" + title  + ".txt"
            if not os.path.exists(output):  # Kiểm tra xem file đã tồn tại chưa
                with open(output, "w+", encoding='utf-8') as f:
                    f.write(title+"\n")
                    f.write(contents)
    except Exception as e:
        print(f"Error processing URL: {url} - {e}")

def vietnamnet(base_url):
    page_urls = get_page_urls_vietnamnet(base_url, quantity=500)
#     print(page_urls)
    total_articles = 0
    for page in page_urls:
        arr = get_links_in_page_vietnamnet(page)
        for i in range(1,len(arr)-1):
            title = arr[i][0]
            url = arr[i][1]
            crawl_by_url(url,title)

# base_url = 'http://vnexpress.net/thoi-su' ## category = thoi-su
# Chạy thêm theo danh sách các thể loại
categories = ['thong-tin-truyen-thong', 'thoi-su', 'the-thao', 'kinh-doanh', 'suc-khoe']
for category in reversed(categories):
    vietnamnet(f'https://vietnamnet.vn/{category}')

In [8]:
import os
import urllib.request
from bs4 import BeautifulSoup
from newspaper import Article

# VIETNAMNET
def get_page_urls_vietnamnet(base_url, quantity):
    news_per_page = 25
    if quantity % news_per_page == 0:
        no_of_pages = quantity // news_per_page
    else:
        no_of_pages = quantity // news_per_page + 1

    extend_url = ['-p{}'.format(page) for page in range(2, no_of_pages + 1)]
    url_list = [base_url + extend_part for extend_part in extend_url]
    url_list.insert(0, base_url)
    return url_list

# VIETNAMNET
def get_links_in_page_vietnamnet(url):
    try:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page, 'html.parser')

        # Tìm tất cả các thẻ h3 với class 'horizontalPost__main-title'
        h3_all = soup.find_all('h3', attrs={'class': 'horizontalPost__main-title'})

        results = []
        for h3 in h3_all:
            a_tag = h3.find('a')
            if a_tag:
                title = a_tag.get('title')
                link = a_tag.get('href')

                # Kiểm tra nếu có link và tiêu đề
                if title and link:
                    # Nếu link là đường dẫn tương đối, thêm base URL vào
                    if link.startswith('/'):
                        link = 'https://vietnamnet.vn' + link
                    print(f'Title: {title} - Link: {link}')
                    results.append([title, link])
        return results
    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e.code} for URL: {url}")
        return []
    except Exception as e:
        print(f"Error processing page: {url} - {e}")
        return []


def crawl_by_url(url, title):
    try:
        article = Article(url, language='vi')
        article.download()
        article.parse()
        contents = str(article.text).replace("\n", "")
        article.nlp()

        # Thay thế các ký tự không hợp lệ trong tiêu đề
        char_to_replace = {
            ':': '', '/': '', '\\': '', '*': '',
            '?': '', '"': '', '<': '', '>': '', '|': ''
        }
        for key, value in char_to_replace.items():
            title = title.replace(key, value)

        # Lưu bài viết vào thư mục
        if 'https://vietnamnet.vn/' in url:
            folder_path = "data/text/vietnamnet/"
            os.makedirs(folder_path, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
            output = os.path.join(folder_path, title + ".txt")

            # Lưu bài viết nếu chưa tồn tại
            if not os.path.exists(output):
                with open(output, "w+", encoding='utf-8') as f:
                    # f.write(str(url) + "\n")
                    f.write(title + "\n")
                    f.write(contents)
    except Exception as e:
        print(f"Error processing URL: {url} - {e}")

def vietnamnet(base_url):
    page_urls = get_page_urls_vietnamnet(base_url, quantity=500)
    for page in page_urls:
        arr = get_links_in_page_vietnamnet(page)
        for article in arr:
            if len(article) == 2:  # Đảm bảo đủ title và URL
                title, url = article
                crawl_by_url(url, title)

# Chạy thêm theo danh sách các thể loại
categories = ['thong-tin-truyen-thong', 'thoi-su', 'the-thao', 'kinh-doanh', 'suc-khoe']
for category in reversed(categories):
    vietnamnet(f'https://vietnamnet.vn/{category}')


Title: Nhiều lần thất bại, cặp vợ chồng vẫn kiên trì tìm con suốt 13 năm - Link: https://vietnamnet.vn/nhieu-lan-that-bai-cap-vo-chong-van-kien-tri-chua-vo-sinh-suot-13-nam-2347888.html
Title: Sinh nhật tuổi 19, chàng trai Gia Lai vào viện tự tặng món quà 'không giống ai' - Link: https://vietnamnet.vn/sinh-nhat-tuoi-19-chang-trai-gia-lai-vao-vien-tu-tang-mon-qua-khong-giong-ai-2341282.html
Title: Vị trí đau lưng cảnh báo thận đầy sỏi - Link: https://vietnamnet.vn/vi-tri-dau-lung-canh-bao-than-day-soi-2347711.html
Title: Hết cơ hội sống sau khi tự chữa ung thư vú - Link: https://vietnamnet.vn/cho-chet-sau-khi-tu-chua-ung-thu-vu-2347817.html
Title: Có khối u nhưng không điều trị, 6 năm sau bị phá hủy xương - Link: https://vietnamnet.vn/co-khoi-u-nhung-khong-dieu-tri-6-nam-sau-bi-pha-huy-xuong-2347856.html
Title: Nữ bác sĩ cạnh tranh với 'tử thần', điều trị cho hơn 100.000 bệnh nhân - Link: https://vietnamnet.vn/nu-bac-si-canh-tranh-voi-tu-than-dieu-tri-cho-hon-100-000-benh-nhan-2344828.h

KeyboardInterrupt: 

In [16]:
import os
import urllib.request
from bs4 import BeautifulSoup
from newspaper import Article

# THANHNIEN
def get_page_urls_thanhnien(base_url, quantity):
    news_per_page = 25
    if quantity % news_per_page == 0:
        no_of_pages = quantity // news_per_page
    else:
        no_of_pages = quantity // news_per_page + 1

    extend_url = ['-p{}'.format(page) for page in range(2, no_of_pages + 1)]
    url_list = [base_url + extend_part for extend_part in extend_url]
    url_list.insert(0, base_url)
    return url_list

# THANHNIEN
def get_links_in_page_thanhnien(url):
    try:
        page = urllib.request.urlopen(url)
        soup = BeautifulSoup(page, 'html.parser')

        # Tìm tất cả các thẻ <a> với class 'box-category-link-title' (cho tiêu đề)
        a_all = soup.find_all('a', attrs={'class': 'box-category-link-title'})

        results = []
        for a in a_all:
            title = a.get('title')
            link = a.get('href')

            # Kiểm tra nếu có link và tiêu đề
            if title and link:
                # Nếu link là đường dẫn tương đối, thêm base URL vào
                if link.startswith('/'):
                    link = 'https://thanhnien.vn' + link
                print(f'Title: {title} - Link: {link}')
                results.append([title, link])
        return results
    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e.code} for URL: {url}")
        return []
    except Exception as e:
        print(f"Error processing page: {url} - {e}")
        return []

def crawl_by_url(url, title):
    try:
        article = Article(url, language='vi')
        article.download()
        article.parse()
        contents = str(article.text).replace("\n", "")
        article.nlp()

        # Thay thế các ký tự không hợp lệ trong tiêu đề
        char_to_replace = {
            ':': '', '/': '', '\\': '', '*': '',
            '?': '', '"': '', '<': '', '>': '', '|': ''
        }
        for key, value in char_to_replace.items():
            title = title.replace(key, value)

        # Lưu bài viết vào thư mục
        folder_path = "data/text/thanhnien/"
        os.makedirs(folder_path, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại
        output = os.path.join(folder_path, title + ".txt")

        # Lưu bài viết nếu chưa tồn tại
        if not os.path.exists(output):
            with open(output, "w+", encoding='utf-8') as f:
                # f.write(str(url) + "\n")
                f.write(title + "\n")
                f.write(contents)
    except Exception as e:
        print(f"Error processing URL: {url} - {e}")

def thanhnien(base_url):
    page_urls = get_page_urls_thanhnien(base_url, quantity=500)
    for page in page_urls:
        arr = get_links_in_page_thanhnien(page)
        for article in arr:
            if len(article) == 2:  # Đảm bảo đủ title và URL
                title, url = article
                crawl_by_url(url, title)

# Chạy thêm theo danh sách các thể loại
# categories = ['van-hoa', 'thoi-su', 'the-gioi', 'kinh-te', 'suc-khoe']
categories = ['chinh-tri']

for category in reversed(categories):
    thanhnien(f'https://thanhnien.vn/{category}.htm')


Title: Để lỡ thời cơ là có lỗi với đất nước, nhân dân - Link: https://thanhnien.vn/de-lo-thoi-co-la-co-loi-voi-dat-nuoc-nhan-dan-185241204003239047.htm
Title: Thủ tướng phân công các Phó thủ tướng phụ trách sắp xếp bộ, ngành - Link: https://thanhnien.vn/thu-tuong-phan-cong-cac-pho-thu-tuong-phu-trach-sap-xep-bo-nganh-185241204095430559.htm
Title: Phương án sắp xếp, sáp nhập 15 bộ, cơ quan ngang bộ của Chính phủ - Link: https://thanhnien.vn/phuong-an-sap-nhap-giam-5-bo-4-co-quan-cua-chinh-phu-185241204084833467.htm
Title: Đưa quan hệ Đối tác chiến lược Việt Nam - Singapore 
lên tầm cao mới - Link: https://thanhnien.vn/dua-quan-he-doi-tac-chien-luoc-viet-nam-singapore-len-tam-cao-moi-185241202224423869.htm
Title: Cuộc cách mạng tinh gọn bộ máy: Không thể chậm trễ hơn nữa - Link: https://thanhnien.vn/cuoc-cach-mang-tinh-gon-bo-may-khong-the-cham-tre-hon-nua-185241201220334799.htm
Title: Đẩy mạnh hợp tác Việt Nam - Nhật Bản trên tất cả các kênh - Link: https://thanhnien.vn/day-manh-hop-tac